# **Plant Seedlings Image Classification CNN - Computer Vision Project _Prabhu_12Feb2021**

#####################################################

### **Context:**
- Can you differentiate a weed from a crop seedling?
- The ability to do so effectively can mean better crop yields and better stewardship of the environment.
- The Aarhus University Signal Processing group, in collaboration with University of Southern Denmark, has recently released a dataset containing images of unique plants belonging to 12 species at several growth stages

### **Objective:**
To implement the techniques learnt as a part of the course.





# **Importing necessary libraries**

In [ ]:
# Import necessary libraries.
import cv2
import numpy as np                               # Import numpy
import pandas as pd                               # Import numpy
import seaborn as sns                             # Import Seaborn
from skimage import data, io                     # Import skimage library (data - Test images and example data, io - Reading, saving, and displaying images) 
import matplotlib.pyplot as plt                  # Import matplotlib.pyplot (Plotting framework in Python.)
%matplotlib inline
import os                                        # This module provides a portable way of using operating system dependent functionality.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import math
from glob import glob
import tensorflow as tf                           # Import tensorflow
from tensorflow.keras.models import Sequential     
from tensorflow.keras.layers import (
    Dense, 
    Dropout, 
    Flatten, 
    Conv2D, 
    MaxPooling2D, 
    MaxPool2D,
    GlobalMaxPooling2D,
    BatchNormalization
)
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import datasets, models, layers, optimizers
from tensorflow.keras.optimizers import RMSprop, Adam       
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical            # convert to one-hot-encoding
from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split       # Import test_train_split from sklearn    
from sklearn.metrics import classification_report, confusion_matrix

import warnings 
warnings.filterwarnings('ignore')        # Suppress warnings               

In [ ]:
# Mount Google drive so dataset can be accessed 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#from zipfile import ZipFile
#with ZipFile(train_path, 'r') as zip:
#  zip.extractall(extract_path)

# **Load dataset, print shape of data, visualize the images in dataset**

In [ ]:
trainLabel = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Labels.csv')
print(trainLabel.shape)

In [ ]:
trainImg = np.load('/content/drive/My Drive/Colab Notebooks/images.npy')

print(trainImg.shape)

In [ ]:
print(f"Training image array shape:{trainImg.shape}")
print(f"Training target labels:{trainLabel.shape}")

In [ ]:
# Check Images
import matplotlib.pyplot as plt
plt.imshow(trainImg[0])

In [ ]:
#sobel = cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=5)
#plt.imshow(sobel)

In [ ]:
# Check Images
import matplotlib.pyplot as plt
plt.imshow(trainImg[500])

In [ ]:
# Check Images
import matplotlib.pyplot as plt
plt.imshow(trainImg[1000])

In [ ]:
# Check Images trainImg [0] -- cv2.Sobel
import matplotlib.pyplot as plt
plt.imshow(trainImg[0])

In [ ]:
#sobel = cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=5)
#plt.imshow(sobel)

#**Pre-processing & Normalizing the data**

In [ ]:
trainImg = trainImg.astype('float32')
trainImg /= 255
# Check the nomalized data
print(f'Shape of the Train array:{trainImg.shape}')
print(f'Minimum value in the Train Array:{trainImg.min()}')
print(f'Maximum value in the Train Array:{trainImg.max()}')

In [ ]:
# Step#1: Split train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainImg, trainLabel, test_size=0.3, random_state=42)  
X_train.shape, X_test.shape

In [ ]:
# Step#2: Split validation from test set 
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
X_test.shape, X_validation.shape

#**One Hot Encoding to Target Values**

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
y_validation = encoder.fit_transform(y_validation)

In [ ]:
# Display target variable
y_train[0]

#**Gaussian Blurring**

In [ ]:
# Preview the image before Gaussian Blur
plt.imshow(X_train[1], cmap='gray')

In [ ]:
plt.imshow(cv2.GaussianBlur(X_train[1], (15,15), 0))

In [ ]:
# Now we apply the gaussian blur to each 128x128 pixels array (image) to reduce the noise in the image
for idx, img in enumerate(X_train):
  X_train[idx] = cv2.GaussianBlur(img, (5, 5), 0)

In [ ]:
# Preview the image after Gaussian Blur
plt.imshow(X_train[0], cmap='gray')

In [ ]:
# Gaussian Blur to Test and Validation sets
for idx, img in enumerate(X_test):
  X_test[idx] = cv2.GaussianBlur(img, (5, 5), 0)

for idx, img in enumerate(X_validation):
  X_validation[idx] = cv2.GaussianBlur(img, (5, 5), 0)

# **Creating  a CNN Model**

Steps:

- Initialize CNN Classifier
- Add Convolution layer with 32 kernels of 3x3 shape
- Add Maxpooling layer of size 2x2
- Flatten the input array
- Add dense layer with relu activation function
- Dropout the probability
- Add softmax Dense layer as output

In [ ]:
def create_model(input_shape, num_classes):
  # Initialize CNN Classified
  model = Sequential()

  # Add convolution layer with 32 filters and 3 kernels
  model.add(Conv2D(32, (3,3), input_shape=input_shape, padding='same', activation=tf.nn.relu))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(rate=0.25))

  # Add convolution layer with 32 filters and 3 kernels
  model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu))
  model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(rate=0.25))

  # Add convolution layer with 32 filters and 3 kernels
  model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(rate=0.25))

  # Flatten the 2D array to 1D array
  model.add(Flatten())

  # Create fully connected layers with 512 units
  model.add(Dense(512, activation=tf.nn.relu))
  model.add(Dropout(0.5))


  # Adding a fully connected layer with 128 neurons
  model.add(Dense(units = 128, activation = tf.nn.relu))
  model.add(Dropout(0.5))

  # The final output layer with 12 neurons to predict the categorical classifcation
  model.add(Dense(units = num_classes, activation = tf.nn.softmax))
  return model

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

es = EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=10)

In [ ]:
input_shape = X_train.shape[1:] # Input shape of X_train
num_classes = y_train.shape[1] # Target column size

model = create_model(input_shape, num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Optimizer
 #optimizer = tf.keras.optimizers.SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=10, batch_size=500, callbacks=[callbacks]) # Initial trial with epochs=10 and batch size=500 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch', fontsize=18)
plt.ylabel(r'Loss', fontsize=18)
plt.legend(('loss train','loss validation'), loc=0)

In [ ]:
# Print accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch', fontsize=18)
plt.ylabel(r'Accuracy', fontsize=18)
plt.legend(('accuracy train','accuracy validation'), loc=0)

**Model Evaluation**

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss: {:.2f} \n Test accuracy: {:.2f}'.format(loss, accuracy))

loss, accuracy = model.evaluate(X_train, y_train)
print('Train loss: {:.2f} \n Train accuracy: {:.2f}'.format(loss, accuracy))

# **Retraining the Model**

- Above try with epochs=10 and batch size=500 resulted in test accuracy = 0.54, validation accuracy of 0.57 which is low and loss is high  -- -shall retry and train model  
- Try and retrain "model1" with different epochs= 30 and batch size = 100 to see if test and validation accurracy increases and loss decreases


In [ ]:
# Retrain as model1
model1 = create_model(input_shape, num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Optimizer

model1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()

In [ ]:
# Above try with epochs=10 and batch size=500  resulted in val accuracy of 0.45 which is not good - shall retry with 
#try with different epochs= 30 and batch size = 100 val accurracy increases significantly to 0.78    or 78% which is good
history = model1.fit(X_train, y_train, validation_data=(X_validation, y_validation), epochs=30, batch_size=100, callbacks=[callbacks])

In [ ]:
# Print Loss ( Model1) -- retrained model
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch', fontsize=18)
plt.ylabel(r'Loss', fontsize=18)
plt.legend(('loss train','loss validation'), loc=0)

In [ ]:
# Print Accuracy ( Model1) -- retrained model
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch', fontsize=18)
plt.ylabel(r'Accuracy', fontsize=18)
plt.legend(('accuracy train','accuracy validation'), loc=0)

**Model Evaluation after re-training (model1)**

In [ ]:
loss, accuracy = model1.evaluate(X_test, y_test)
print('Test loss: {:.2f} \n Test accuracy: {:.2f}'.format(loss, accuracy))

loss, accuracy = model1.evaluate(X_train, y_train)
print('Train loss: {:.2f} \n Train accuracy: {:.2f}'.format(loss, accuracy))

- From above retained "model1" test accuracy = 0.82 validation accuracy = 0.96 increased & loss decreased --- which is very good

# **Confusion Matrix**

In [ ]:
y_pred = model1.predict(X_test)
y_pred = (y_pred > 0.5) 

In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
print("=== Classification Report ===")
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

- Precision: Out of all the positive classes we have predicted correctly, how many are actually positive.
- Recall: Out of all the positive classes, how much we predicted correctly. It should be high as possible.
- F1-Score: F1 Score is the weighted average of Precision and Recall. 

Therefore, this score takes both false positives and false negatives into account. Intuitively it is not as easy to understand as accuracy, but F1 is usually more useful than accuracy, especially if you have an uneven class distribution

# **Visualize predictions for x_test[2], x_test[3], x_test[33], x_test[36], x_test[59]**

In [ ]:
y_pred = encoder.inverse_transform(y_pred)

index = 2
plt.imshow(X_test[index], cmap='gray')
print("Predicted label:", y_pred[index])

In [ ]:
index = 3
plt.imshow(X_test[index], cmap='gray')
print("Predicted label:", y_pred[index])

In [ ]:
index = 33
plt.imshow(X_test[index], cmap='gray')
print("Predicted label:", y_pred[index])

In [ ]:
index = 36
plt.imshow(X_test[index], cmap='gray')
print("Predicted label:", y_pred[index])

In [ ]:
index = 59
plt.imshow(X_test[index], cmap='gray')
print("Predicted label:", y_pred[index])

**Model Prediction**

In [ ]:
y_pred    # Model prediction below array shows all predicted species ( seedlings and weeds)

#  Above "y_pred" array shows all predicted species (seedlings and weeds)




********************************************************************************

# **All below project steps and tasks were achieved sucessfully**
1. Import the libraries, load dataset, print shape of data, visualize the images in dataset. (5 Marks)
2. Data Pre-processing: (15 Marks)
a. Normalization.
b. Gaussian Blurring.
c. Visualize data after pre-processing.
3. Make data compatible: (10 Marks)
a. Convert labels to one-hot-vectors.
b. Print the label for y_train[0].
c. Split the dataset into training, testing, and validation set.
(Hint: First split images and labels into training and testing set with test_size = 0.3. Then further split test data
into test and validation set with test_size = 0.5)
d. Check the shape of data, Reshape data into shapes compatible with Keras models if it’s not already. If it’s
already in the compatible shape, then comment in the notebook that it’s already in compatible shape.
4. Building CNN: (15 Marks)
a. Define layers.
b. Set optimizer and loss function. (Use Adam optimizer and categorical crossentropy.)
5. Fit and evaluate model and print confusion matrix. (10 Marks)
6. Visualize predictions for x_test[2], x_test[3], x_test[33], x_test[36], x_test[59]. (5 Marks)


********************************************************************************